In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the directory path in Google Drive
model_directory = '/content/drive/MyDrive/bert_model'

# Load the BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained(model_directory)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define maximum token limit for BERT
max_length = model.config.max_position_embeddings

# Load data from CSV file
csv_file_path = '/content/drive/MyDrive/dataset_small.csv'
df = pd.read_csv(csv_file_path, encoding='ISO-8859-1')




Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [2]:
# Convert date column to datetime format (optional but recommended)
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
# Keep only the first 100 rows for each date
def filter_by_date(df, n=100):
  return df.groupby('date').head(n)

result_df = filter_by_date(df.copy(), n=100)

In [3]:
df=result_df.copy()

In [4]:
# Define a function to make predictions
def predict(text):
  try:
      inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Check for presence of OOV words
      # if tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]) != text.split():  # Assuming tokenization splits on whitespace
      #   print(f"WARNING: Text contains OOV words. Using special tokens.")
      # print(len(inputs))
      outputs = model(**inputs)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=1).cpu().numpy()
      return predictions[0]
  except Exception as e:
        # Print the error value for debugging
        #print(f"Error during prediction for text: {text}\nError message: {e}")
        return None  # Or any desired default value


In [5]:
# Make predictions and write results back to the DataFrame
df['Sentiment'] = df['news'].apply(predict)

In [6]:
# Save the updated DataFrame back to CSV
output_csv_path = '/content/drive/MyDrive/data_with_predictions.csv'
df.to_csv(output_csv_path, index=False)

print(f"Predictions saved to {output_csv_path}")


Predictions saved to /content/drive/MyDrive/data_with_predictions.csv
